In [23]:
import pickle
import re
import string
import nltk
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer


In [11]:
subs = ['wsb', 'science', 'ama', 'askreddit', 'worldnews', 'funny', 'dankmemes', 'memes']

# Load pickled files
data = {}
for i, c in enumerate(subs):
    with open("/home/nick/Documents/Reddit_NLP/" + c + ".pickle", "rb") as file:
        data[c] = pickle.load(file)

In [12]:
data.keys()

dict_keys(['wsb', 'science', 'ama', 'askreddit', 'worldnews', 'funny', 'dankmemes', 'memes'])

In [3]:
len(wsb) + len(science) + len(ama) + len(askreddit) + len(worldnews) + len(funny) + len(dankmemes) + len(memes)

151679

In [14]:
# Turn lists of comments into one big string
def combine_text(comments):
    combined_text = ' '.join(comments)
    return combined_text
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [24]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['text']
data_df = data_df.sort_index()
data_df

,text
ama,Tldr: do whatever the fuck you want. So you're telling me people on the internet LIE???? [removed] This is so funny this subreddit got fucked by o...
askreddit,"Victoria’s Secret was originally supposed to be a place for men to buy lingerie for their wives and not feel embarrassed, but now it’s almost excl..."
dankmemes,"Meme of the Month Seeding and Nominations for July 2020!\n\n# How do nominations work?\n\nTo nominate a meme, simply comment a name of a meme that..."
funny,"But I wanted the robot ferret. Haha! Suck it you fools, I already got my robotic ferret from a mod! This not funny!\nRule number...... Oh, I forge..."
memes,"RedditMC is opening staff positions: Builders, Helpers (To become mod), Devs, GameMasters...\n\nMore infos [here](https://www.reddit.com/r/RedditM..."
science,Am I reading this correctly to conclude that this research supports the emergent theory of consciousness? It's insane to see the advances this has...
worldnews,"So was there any posts left after the purge? Ok, just for some context:\n\nFacebook has an estimated ~100 million fake user accounts.\nEvery month..."
wsb,after 8 green days in a row...\n\nWSB: wtf where did that sell off come from??! Man remember 1pm? Those were the days Breaking: **Silver in talks ...


In [16]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [17]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

,transcript
ama,tldr do whatever the fuck you want so youre telling me people on the internet lie this is so funny this subreddit got fucked by one person a note...
askreddit,victoria’s secret was originally supposed to be a place for men to buy lingerie for their wives and not feel embarrassed but now it’s almost exclu...
dankmemes,meme of the month seeding and nominations for july \n\n how do nominations work\n\nto nominate a meme simply comment a name of a meme that started...
funny,but i wanted the robot ferret haha suck it you fools i already got my robotic ferret from a mod this not funny\nrule number oh i forget carry on b...
memes,redditmc is opening staff positions builders helpers to become mod devs gamemasters\n\nmore infos using this to see if i have new flair cries in ...
science,am i reading this correctly to conclude that this research supports the emergent theory of consciousness its insane to see the advances this has m...
worldnews,so was there any posts left after the purge ok just for some context\n\nfacebook has an estimated million fake user accounts\nevery month about ...
wsb,after green days in a row\n\nwsb wtf where did that sell off come from man remember those were the days breaking silver in talks to buy tiktok t...


In [20]:
# Get rid of newlines, punctuation that was missed
def clean_text_round2(text):
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [21]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_clean

,transcript
ama,tldr do whatever the fuck you want so youre telling me people on the internet lie this is so funny this subreddit got fucked by one person a note...
askreddit,victorias secret was originally supposed to be a place for men to buy lingerie for their wives and not feel embarrassed but now its almost exclusi...
dankmemes,meme of the month seeding and nominations for july how do nominations workto nominate a meme simply comment a name of a meme that started or peak...
funny,but i wanted the robot ferret haha suck it you fools i already got my robotic ferret from a mod this not funnyrule number oh i forget carry on but...
memes,redditmc is opening staff positions builders helpers to become mod devs gamemastersmore infos using this to see if i have new flair cries in bedr...
science,am i reading this correctly to conclude that this research supports the emergent theory of consciousness its insane to see the advances this has m...
worldnews,so was there any posts left after the purge ok just for some contextfacebook has an estimated million fake user accountsevery month about billio...
wsb,after green days in a rowwsb wtf where did that sell off come from man remember those were the days breaking silver in talks to buy tiktok this ...
